In [1]:
EEG_channels = 22 #from project guidelines
test_count = 50 #from project guideline, 238 for train-validation and 50 for test
validation_count = 38 # 38 points in validation set and remaining 200 points in test set

In [2]:
from includes import *

#################### "includes" imports:
#
#   from read_data import *
#   
#   import torch
#   from torch.autograd import Variable
#   import torch.nn as nn
#   import torch.optim as optim
#
#   dtype = torch.cuda.FloatTensor # torch.FloatTensor
#
#   all_files = [h5py.File(m, 'r') for m in mat_names]
#   all_ims = [f['image'] for f in all_files]
#   all_types = [f['type'] for f in all_files]
#
####################
%load_ext autoreload
%autoreload 2

In [3]:
image_mat = all_ims[0]
image_shape = image_mat.shape # 288 (48x6) trials across 25 electrodes for 1000 time points (250Hz*4s)
print image_shape

(288, 25, 1000)


In [4]:
print(mat_names)

['./project_datasets/A01T_slice.mat', './project_datasets/A02T_slice.mat', './project_datasets/A03T_slice.mat', './project_datasets/A04T_slice.mat', './project_datasets/A05T_slice.mat', './project_datasets/A06T_slice.mat', './project_datasets/A07T_slice.mat', './project_datasets/A08T_slice.mat', './project_datasets/A09T_slice.mat']


In [5]:
#setting seed
np.random.seed(seed=1337)
test_picked = np.random.choice(image_shape[0], test_count, replace=False)
train_val_picked = np.setdiff1d(np.arange(image_shape[0]), test_picked)
val_picked = train_val_picked[:validation_count]
train_picked = train_val_picked[validation_count:]

In [6]:
type_mat = all_types[0]
type_shape = type_mat.shape
type_set = list(set(type_mat[0,:]))

# all the 0's occur after 288, and are meaningless I think
# so the image_mat, which has shape (288, 25, 1000) should correspond
# to the first 288 entries of type_mat, so
# for a single subject, training data should be image_mat, with 288 samples, each sample has shape (25, 1000)
# and our target label matrix should be type_mat[:288] (or 287?)

In [7]:
trainval_data_X = []
training_data_X = []
validation_data_X = []
test_data_X = []

trainval_data_Y = []
training_data_Y = []
validation_data_Y = []
test_data_Y = []

for i in range(len(mat_names)):
    matfile = h5py.File(mat_names[i], 'r')
    
    trainval_data_X.append(matfile['image'][sorted(train_val_picked),:EEG_channels,:]) #(238, 22, 1000) x 9
    training_data_X.append(matfile['image'][sorted(train_picked),:EEG_channels,:]) #(200, 22, 1000) x 9
    validation_data_X.append(matfile['image'][sorted(val_picked),:EEG_channels,:]) #(38, 22, 1000) x 9
    test_data_X.append(matfile['image'][sorted(test_picked),:EEG_channels,:]) #(50, 22, 1000) x 9
    
    trainval_data_Y.append(matfile['type'][0,sorted(train_val_picked)] - type_set[1]) #(238, ) x 9
    training_data_Y.append(matfile['type'][0,sorted(train_picked)] - type_set[1]) #(200, ) x 9
    validation_data_Y.append(matfile['type'][0,sorted(val_picked)] - type_set[1]) #(38, ) x 9
    test_data_Y.append(matfile['type'][0,sorted(test_picked)] - type_set[1]) #(50, ) x 9

In [8]:
for i in range(len(mat_names)):
    assert(set(training_data_Y[i]) == set(validation_data_Y[i])) #check whether we have data for all labels 
    assert(set(training_data_Y[i]) == set(test_data_Y[i])) #check whether we have data for all labels 

In [9]:
from functools import reduce

rnn_trainval_data_X = np.concatenate(trainval_data_X, axis=0) #(2142, 22, 1000)
rnn_training_data_X = np.concatenate(training_data_X, axis=0) #(1800, 22, 1000)
rnn_validation_data_X = np.concatenate(validation_data_X, axis=0) #(342, 22, 1000)
rnn_test_data_X = np.concatenate(test_data_X, axis=0) #(450, 22, 1000)

rnn_trainval_data_Y = np.concatenate(trainval_data_Y, axis=0) #(2142, )
rnn_training_data_Y = np.concatenate(training_data_Y, axis=0) #(1800, )
rnn_validation_data_Y = np.concatenate(validation_data_Y, axis=0) #(342, )
rnn_test_data_Y = np.concatenate(test_data_Y, axis=0) #(450,)

def remove_nan_rows_A(A, b):
    if (np.isnan(A).any() or np.isnan(b).any()):
        mask = ~np.isnan(np.sum(A,axis=(1,2))) & ~np.isnan(b[:])
        A = A[mask, :, :]
        b = b[mask]
        
    assert A.shape[0] == b.shape[0]
    return A, b

rnn_trainval_data_X, rnn_trainval_data_Y = remove_nan_rows_A(rnn_trainval_data_X,
                                                             rnn_trainval_data_Y)
rnn_training_data_X, rnn_training_data_Y = remove_nan_rows_A(rnn_training_data_X, 
                                                             rnn_training_data_Y)
rnn_validation_data_X, rnn_validation_data_Y = remove_nan_rows_A(rnn_validation_data_X,
                                         rnn_validation_data_Y)
rnn_test_data_X, rnn_test_data_Y = remove_nan_rows_A(rnn_test_data_X,
                                   rnn_test_data_Y)


# repeating the Y labels for the rnn
N_trainval, E, T = rnn_trainval_data_X.shape
N_training, _, _ = rnn_trainval_data_X.shape
N_validation, _, _ = rnn_test_data_X.shape
N_test, _, _ = rnn_test_data_X.shape

In [10]:
print(training_data_Y[0].shape)

(200,)


In [11]:
training_data_shape = training_data_X[0].shape
print(training_data_shape) #(200, 22, 1000) while test data shape is (50, 22, 1000) and validation data is (38, 22,1000)

(200, 22, 1000)


In [12]:
print rnn_trainval_data_X.shape
print rnn_trainval_data_Y.shape

(2115, 22, 1000)
(2115,)


In [13]:
from torch.nn.utils.rnn import pack_padded_sequence as ppseq

In [14]:
tensor_x_train = torch.Tensor(rnn_training_data_X)
tensor_y_train = torch.LongTensor(rnn_training_data_Y)

print tensor_x_train.shape
print tensor_y_train.shape

tensor_x_val = torch.Tensor(rnn_validation_data_X)
tensor_y_val = torch.LongTensor(rnn_validation_data_Y)

torch.Size([1775, 22, 1000])
torch.Size([1775])


In [15]:
import torch.utils.data

In [16]:
# Hyper Parameters
num_epochs = 20
batch_size = 100
learning_rate = 1e-1

In [17]:
train_dataset = torch.utils.data.TensorDataset(tensor_x_train,tensor_y_train)
val_dataset = torch.utils.data.TensorDataset(tensor_x_val,tensor_y_val)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [24]:
from VanillaRNN import VanillaRNN

NUM_CLASSES = 4
net = VanillaRNN(recurrent_hidden_size=100, num_classes=NUM_CLASSES)

In [25]:
net.train(True)
out, hidden = net.forward(tensor_x_train)
print(out.shape)
net.train(False)
out, hidden = net.forward(tensor_x_train)
print(out.shape)

torch.Size([1775000, 4])
torch.Size([1775, 4])


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
train_acc = []
val_acc = []
loss_history = []
          
for epoch in range(num_epochs):

    ######################## TRAINING
        
    net.train(True)
    
    for i, (images, labels) in enumerate(train_loader):
        
        #     for i in range(cnn_training_data_X.shape[0]):
        images = Variable(images, requires_grad=True) #unsqueeze used to make a 4d tensor because 
        #     print images.shape
        labels = labels.repeat(T)
        labels = Variable(labels)

        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs, hidden = net(images)
        
        #reshaped_outputs = outputs.view(-1, NUM_CLASSES)
        reshaped_labels = labels.view(-1)
        
        loss = criterion(outputs, reshaped_labels)
        loss_history.append(loss)
        loss.backward()
        optimizer.step()

        print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
               %(epoch+1, num_epochs, i+1, rnn_training_data_X.shape[0]*1.0/batch_size, loss.data[0]))
    
    ######################## PER EPOCH EVALUATION
    net.train(False)
    
    images = Variable(torch.Tensor(rnn_training_data_X))
    test_labels = torch.LongTensor(rnn_training_data_Y)
    outputs, hidden = net(images)
    correct = 0
    total = 0
    _, predicted = torch.max(outputs.data, 1)
    total += test_labels.size(0)
    correct += (predicted == test_labels).double().sum()
    print('Train Accuracy: %d %%' % (100 * correct / total))
    train_acc.append((100 * correct / total))

    net.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
    correct = 0
    total = 0
    images = Variable(torch.Tensor(rnn_validation_data_X))
    test_labels = torch.LongTensor(rnn_validation_data_Y)
    outputs, hidden = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += test_labels.size(0)
    correct += (predicted == test_labels).double().sum()
    print('validation Accuracy: %d %%' % (100.0 * correct / total))
    val_acc.append((100 * correct / total))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:32: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch [1/20], Iter [1/17] Loss: 1.4513
Epoch [1/20], Iter [2/17] Loss: 1.4510
Epoch [1/20], Iter [3/17] Loss: 1.4518
Epoch [1/20], Iter [4/17] Loss: 1.4474
Epoch [1/20], Iter [5/17] Loss: 1.4498
Epoch [1/20], Iter [6/17] Loss: 1.4465
Epoch [1/20], Iter [7/17] Loss: 1.4451
Epoch [1/20], Iter [8/17] Loss: 1.4481
Epoch [1/20], Iter [9/17] Loss: 1.4491
Epoch [1/20], Iter [10/17] Loss: 1.4433
Epoch [1/20], Iter [11/17] Loss: 1.4476
Epoch [1/20], Iter [12/17] Loss: 1.4463
Epoch [1/20], Iter [13/17] Loss: 1.4438
Epoch [1/20], Iter [14/17] Loss: 1.4441
Epoch [1/20], Iter [15/17] Loss: 1.4456
Epoch [1/20], Iter [16/17] Loss: 1.4411
Epoch [1/20], Iter [17/17] Loss: 1.4468
Epoch [1/20], Iter [18/17] Loss: 1.4445
Train Accuracy: 24 %
validation Accuracy: 20 %
Epoch [2/20], Iter [1/17] Loss: 1.4422
Epoch [2/20], Iter [2/17] Loss: 1.4393
Epoch [2/20], Iter [3/17] Loss: 1.4412
Epoch [2/20], Iter [4/17] Loss: 1.4429
Epoch [2/20], Iter [5/17] Loss: 1.4399
Epoch [2/20], Iter [6/17] Loss: 1.4371
Epoch [2

Epoch [11/20], Iter [16/17] Loss: 1.4070
Epoch [11/20], Iter [17/17] Loss: 1.4052
Epoch [11/20], Iter [18/17] Loss: 1.4083
Train Accuracy: 25 %
validation Accuracy: 19 %
Epoch [12/20], Iter [1/17] Loss: 1.4023
Epoch [12/20], Iter [2/17] Loss: 1.4043
Epoch [12/20], Iter [3/17] Loss: 1.4106
Epoch [12/20], Iter [4/17] Loss: 1.4056
Epoch [12/20], Iter [5/17] Loss: 1.4049
Epoch [12/20], Iter [6/17] Loss: 1.4057
Epoch [12/20], Iter [7/17] Loss: 1.4065
Epoch [12/20], Iter [8/17] Loss: 1.4050
Epoch [12/20], Iter [9/17] Loss: 1.4077
Epoch [12/20], Iter [10/17] Loss: 1.4050
Epoch [12/20], Iter [11/17] Loss: 1.4039
Epoch [12/20], Iter [12/17] Loss: 1.4074
